In [ ]:
import pandas as pd
import numpy as np 
import PIL 
import os
from PIL import Image
import json 
import sys 
 
from google.colab.patches import cv2_imshow

import torch
import torchvision
from torchvision import transforms
from torchvision import datasets 
from torch.optim import lr_scheduler

import matplotlib.pyplot as plt 



In [ ]:
!pip install timm 

from timm import create_model 

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

In [ ]:
from torchvision.transforms.transforms import Resize
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
    transforms.Resize((299,299)),
    transforms.RandomHorizontalFlip(),                  
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]),    
    'val': transforms.Compose([
    transforms.Resize((299,299)),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
}




In [ ]:
data_dir = '/content/drive/MyDrive/hymenoptera_data'


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}



dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}




/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [ ]:
class_names

['ants', 'bees']

In [ ]:
# def imshow(inp, title=None):
#     """Imshow for Tensor."""
#     inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)  # pause a bit so that plots are updated


# # Get a batch of training data
# inputs, classes = next(iter(dataloaders['train']))

# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])

In [ ]:
model_name = "convnext_large"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device = ", device)
model = create_model(model_name, pretrained=True).to(device)

In [ ]:
# import torch.nn as nn 

# classifier = nn.Sequential(
#   nn.Linear(in_features=2048, out_features=1024),
#   nn.ReLU(),
#   nn.Dropout(p=0.4),
#   nn.Linear(in_features=1024, out_features=64),
#   nn.LogSoftmax(dim=1)  
# )

In [ ]:
import time
import copy 

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# def visualize_model(model, num_images=6):
#     was_training = model.training
#     model.eval()
#     images_so_far = 0
#     fig = plt.figure()

#     with torch.no_grad():
#         for i, (inputs, labels) in enumerate(dataloaders['val']):
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             outputs = model(inputs)
#             _, preds = torch.max(outputs, 1)

#             for j in range(inputs.size()[0]):
#                 images_so_far += 1
#                 ax = plt.subplot(num_images//2, 2, images_so_far)
#                 ax.axis('off')
#                 ax.set_title('predicted: {}'.format(class_names[preds[j]]))
#                 imshow(inputs.cpu().data[j])

#                 if images_so_far == num_images:
#                     model.train(mode=was_training)
#                     return
#         model.train(mode=was_training)

In [ ]:
import torch.nn as nn 
import torch.optim as optim 
# model.num_features = nn.Linear(model.num_features, 2)

In [ ]:
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(resnet.parameters(), lr = 0.0001, momentum  = 0.7 )
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

In [ ]:
from torchvision import models 

model = models.resnet50(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
from torch.nn.modules import dropout
from torchvision import models

incept = models.inception_v3(pretrained=True).to(device)
incept.aux_logits = False 
# incept.AuxLogits = False

for param in incept.parameters() : 
  param.requires_grad = False

incept.fc =nn.Sequential(
               
               nn.Linear(incept.fc.in_features, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device) 



Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

In [ ]:
incept.fc

Sequential(
  (0): Linear(in_features=2048, out_features=128, bias=True)
  (1): ReLU(inplace=True)
  (2): Linear(in_features=128, out_features=2, bias=True)
)

In [ ]:
squeezenet = models.squeezenet1_0(pretrained =  True ).to(device)

squeezenet.aux_logits = False 

for param in squeezenet.parameters() : 
  param.requires_grad = True

squeezenet.fc = nn.Sequential(
               nn.Linear(2048, 1024),
               nn.Dropout(p=0.5),
               nn.ReLU(inplace = True ), 
               nn.Linear(1024,128), 
               nn.Dropout(p=0.5),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)


Downloading: "https://download.pytorch.org/models/squeezenet1_0-b66bff10.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_0-b66bff10.pth


  0%|          | 0.00/4.78M [00:00<?, ?B/s]

In [ ]:
shufflenet = models.shufflenet_v2_x0_5(pretrained = True ).to(device)

shufflenet.aux_logits = False 

for param in shufflenet.parameters() : 
  param.requires_grad = False 


shufflenet.fc = nn.Sequential(
               nn.Linear(1024, 512),
               nn.Dropout(p=0.5),
               nn.ReLU(inplace = True ), 
               nn.Linear(512,256), 
               nn.Dropout(p=0.5),
               nn.ReLU(inplace=True),
               nn.Linear(256, 2)).to(device)

Downloading: "https://download.pytorch.org/models/shufflenetv2_x0.5-f707e7126e.pth" to /root/.cache/torch/hub/checkpoints/shufflenetv2_x0.5-f707e7126e.pth


  0%|          | 0.00/5.28M [00:00<?, ?B/s]

In [ ]:
optimizer1 = torch.optim.RMSprop(filter(lambda p: p.requires_grad, incept.parameters()), lr=0.001, weight_decay= 1e-4)
criterion = nn.CrossEntropyLoss() 
exp_lr_scheduler = lr_scheduler.StepLR(optimizer1, step_size=7, gamma=0.1)


In [ ]:
# resnet,
model = train_model(incept, criterion, optimizer1, exp_lr_scheduler, num_epochs=70 )

In [ ]:
torch.save(model, '/content/model_incept_new.pt')

In [ ]:
!cp /content/model_incept_new.pt /content/drive/MyDrive/pytorch_models

In [ ]:

import torch.nn as nn
import torch


# class MyEnsemble(nn.Module):

#     def __init__(self, modelA, modelB, modelC, input):
#         super(MyEnsemble, self).__init__()
#         self.modelA = modelA
#         self.modelB = modelB
#         self.modelC = modelC

#         self.fc1 = nn.Linear(input, 16)

#     def forward(self, x):
#         out1 = self.modelA(x)
#         out2 = self.modelB(x)
#         out3 = self.modelC(x)

#         out = out1 + out2 + out3

#         x = self.fc1(out)
#         return torch.softmax(x, dim=1)


import torch.nn.functional as f 

class MyEnsemble(nn.Module):
    def __init__(self, modelA, modelB, modelC , nb_classes=10):
        super(MyEnsemble, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.modelC = modelC 
        # Remove last linear layer
        self.modelA.fc = nn.Identity()
        self.modelB.fc = nn.Identity()
        self.modelC.fc = nn.Identity() 
        
        # Create new classifier

        self.classifier = nn.Linear(6144, nb_classes)
        
    def forward(self, x):
        x1 = self.modelA(x.clone())  # clone to make sure x is not changed by inplace methods
        x1 = x1.view(x1.size(0), -1)
        x2 = self.modelB(x)
        x2 = x2.view(x2.size(0), -1)
        x3 = self.modelC(x)
        x3 = x3.view(x2.size(0), -1)

        x = torch.cat((x1, x2, x3), dim=1)
        
        x = self.classifier(f.relu(x))
        return x

In [ ]:
import torch 
modelA = models.inception_v3(pretrained = True ).to(device) 

for param in modelA.parameters(): 
  param.required_grads = False 

modelA.aux_logits = False 

modelA.fc = nn.Sequential(
    
    nn.Linear(modelA.fc.in_features , 512 ), 
    nn.Dropout(p = 0.2 ), 
    nn.ReLU(inplace = True),
    nn.Linear(512, 128),
    nn.Dropout(p = 0.2 ), 
    nn.ReLU(inplace = True),
    nn.Linear(128,2) ).to(device)


In [ ]:
import torch 
modelB = models.inception_v3(pretrained = True ).to(device) 

for param in modelB.parameters(): 
  param.required_grads = False 

modelB.aux_logits = False 

modelB.fc = nn.Sequential(
    
    nn.Linear(modelB.fc.in_features , 512 ), 
    nn.Dropout(p = 0.2 ), 
    nn.ReLU(inplace = True),
    nn.Linear(512, 128),
    nn.Dropout(p = 0.2 ), 
    nn.ReLU(inplace = True),
    nn.Linear(128,2) ).to(device)


import torch 
modelC = models.inception_v3(pretrained = True ).to(device) 

for param in modelC.parameters(): 
  param.required_grads = False 

modelC.aux_logits = False 

modelC.fc = nn.Sequential(
    
    nn.Linear(modelC.fc.in_features , 512 ), 
    nn.Dropout(p = 0.2 ), 
    nn.ReLU(inplace = True),
    nn.Linear(512, 128),
    nn.Dropout(p = 0.2 ), 
    nn.ReLU(inplace = True),
    nn.Linear(128,2) ).to(device)



In [ ]:
model_final = MyEnsemble(modelA, modelB, modelC, 2).to(device)

In [ ]:
model = train_model(model_final, criterion, optimizer1, exp_lr_scheduler, num_epochs= 80 )

Epoch 0/80
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train Loss: 0.7138 Acc: 0.5123
val Loss: 0.7257 Acc: 0.4183

Epoch 1/80
----------
train Loss: 0.7136 Acc: 0.4959
val Loss: 0.7210 Acc: 0.4771

Epoch 2/80
----------
train Loss: 0.7219 Acc: 0.5615
val Loss: 0.7136 Acc: 0.5033

Epoch 3/80
----------
train Loss: 0.7293 Acc: 0.4754
val Loss: 0.7236 Acc: 0.4379

Epoch 4/80
----------
train Loss: 0.7256 Acc: 0.4877
val Loss: 0.7179 Acc: 0.4706

Epoch 5/80
----------
train Loss: 0.7298 Acc: 0.4795
val Loss: 0.7120 Acc: 0.4902

Epoch 6/80
----------
train Loss: 0.7369 Acc: 0.4672
val Loss: 0.7134 Acc: 0.4379

Epoch 7/80
----------
train Loss: 0.7161 Acc: 0.4877
val Loss: 0.7202 Acc: 0.4771

Epoch 8/80
----------
train Loss: 0.7190 Acc: 0.5041
val Loss: 0.7172 Acc: 0.4575

Epoch 9/80
----------
train Loss: 0.7045 Acc: 0.5041
val Loss: 0.7178 Acc: 0.4771

Epoch 10/80
----------
train Loss: 0.7044 Acc: 0.5287
val Loss: 0.7246 Acc: 0.4641

Epoch 11/80
----------
train Loss: 0.6925 Acc: 0.5287
val Loss: 0.7084 Acc: 0.4641

Epoch 12/80
----------
t

In [ ]:
import torch  
torch.save(model,'/content/model_incept_ensembled_1.pt')

!cp /content/model_incept_ensembled_1.pt /content/drive/MyDrive/pytorch_models

In [ ]:
test_transforms = transforms.Compose([
                                       
                                       transforms.Resize((299,299)),
                                       transforms.ToTensor() 
])


In [ ]:
from torch.autograd import Variable 
def predict_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index

In [ ]:
model_ = torch.load('/content/drive/MyDrive/pytorch_models/model_incept_new.pt')

type(model_)

In [ ]:
model = torch.load('/content/drive/MyDrive/pytorch_models/model_incept_ensembled_1.pt')
type(model)

In [ ]:
import matplotlib.pyplot as plt 
import cv2 
from google.colab.patches import cv2_imshow 


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

to_pil = transforms.ToPILImage()
image = cv2.imread('/content/drive/MyDrive/hymenoptera_data/train/ants/1099452230_d1949d3250.jpg')
cv2_imshow(image)
image = to_pil(image)

image_tensor = test_transforms(image).float()
image_tensor = image_tensor.unsqueeze_(0)
input = Variable(image_tensor)
input = input.to(device)
index = predict_image(image)

if index == 0  :
  print('it is an ant')
elif index == 1 : 
  print('it is a bee')
else : 
  print('Wrong input image')




In [ ]:
classes = ['IT IS AN ANT' , 'IT IS A BEE'] 

In [ ]:
classes[1]

'IT IS A BEE'